In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='NONE' 
SS_VER='SS_VER_1_1'

In [ ]:
# Adapt this to your ChipWhisperer installation
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
GADGET = "ZERO"  # possible values are ["AB", "BA", "AND", "ADD", "ZERO"]
ARCH = 'naive_asm' # possible values are ["c", "asm", "naive_asm"]

if ARCH == 'asm':
    samples = {"AB": 1200, "BA": 600, "AND": 600, "ADD": 1100, "ZERO": 1200}
elif ARCH == 'naive_asm':
    samples = {"AB": 600, "BA": 300, "AND": 300, "ADD": 700, "ZERO": 600}
elif ARCH == 'c':
    samples = {"AB": 500, "BA": 200, "AND": 300, "ADD": 500, "ZERO": 500}

randomness = {"AB":2, "BA": 2, "AND": 2, "ADD": 6, "ZERO": 18}
RAND = randomness[GADGET]
scope.adc.samples = samples[GADGET]

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$GADGET" "$ARCH" "$RAND"
cd target
make clean PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 GADGET=$4 ARCH=$5 RAND=$6 -j

In [ ]:
cw.program_target(scope, prog, "target/frodokem-{}.hex".format(PLATFORM))

In [ ]:
VERBOSE = False
nb_traces = 5000
traces_1, traces_2, x = get_traces(nb_traces)

In [ ]:
%matplotlib ipympl
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
from scipy.stats import norm

alpha = 10**(-5) # error rate
sigma = 1 - (1 - alpha)**(1/samples[GADGET])
th = norm.ppf(1 - sigma/2) # we adapt the threshold depending on number of samples

t_val = ttest_ind(traces_1, traces_2, axis=0, equal_var=False)[0]
plt.close()
plt.plot(t_val[0:], linewidth=0.5)
plt.plot([th]*scope.adc.samples, color = 'r')
plt.plot([-th]*scope.adc.samples, color='r')
plt.show()

In [ ]:
from random import randint
from tqdm.notebook import trange
from secrets import randbits

N_SHARES=2

def get_traces(N):
    traces_1 = []
    traces_2 = []
    x = []
    FIXED = 42
    
    for i in trange(N, desc='Capturing traces'):
        scope.arm()

        mask = randbits(16)

        if GADGET == "AB":
            x0 = (FIXED + mask)%2**16
            x1 = (- mask) % 2**16
        elif GADGET in ["BA", "AND", "ADD", "ZERO"]:
            x0 = FIXED ^ mask
            x1 = mask

        fixed_text = bytearray([x0>>8, x0%256, x1>>8, x1%256])

        if GADGET in ["AND", "ADD"]:
            mask = randbits(16)
            y0 = FIXED ^ mask
            y1 = mask
            fixed_text.extend([y0>>8, y0%256, y1>>8, y1%256])

        randomness = bytearray([randbits(8), randbits(8)])
        if GADGET == "ADD":
            randomness.extend([randbits(8) for _ in range(4)])
        elif GADGET == "ZERO":
            randomness.extend([randbits(8) for _ in range(16)])
        fixed_text.extend(randomness)
        
        if randbits(1):
            if GADGET == "ADD":
                val = (x0^x1) + (y0^y1)
            elif GADGET == "AND":
                val = (x0^x1) & (y0^y1)
            elif GADGET == "ZERO":
                val = (FIXED == 0)*1
            else:
                val = FIXED

            target.simpleserial_write('p', fixed_text)
             
            ret = scope.capture()
            t = scope.adc.trig_count
            if ret:
                print("Target timed out!")
            response = target.simpleserial_read('r', N_SHARES)
            traces_1.append(scope.get_last_trace())
        else:
            text = bytearray([randbits(8) for _ in range(len(fixed_text))])
            if GADGET == "AB":
                val = ((text[0] << 8 | text[1]) + (text[2] << 8 | text[3]))%2**16
            elif GADGET in ["BA", "AND", "ADD", "ZERO"]:
                val = ((text[0] << 8 | text[1]) ^ (text[2] << 8 | text[3]))
            if GADGET == "AND":
                val = val & ((text[4] << 8 | text[5]) ^ (text[6] << 8 | text[7]))
            elif GADGET == "ADD":
                val = (val + ((text[4] << 8 | text[5]) ^ (text[6] << 8 | text[7])))%2**16
            elif GADGET == "ZERO":
                val = (val == 0)*1
    
            target.simpleserial_write('p', text)
            ret = scope.capture()
            if ret:
                print("Target timed out!")
            response = target.simpleserial_read('r', N_SHARES)
            traces_2.append(scope.get_last_trace())

        if VERBOSE:
            print(val == (int(response[0])<<8) + int(response[1]))
            
    size = min(len(traces_1), len(traces_2))
    
    print("Trigger up for", t, "ADC cycles")
    return (traces_1[:size], traces_2[:size], x)

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt

plt.close()
for t in traces_1[:10]:
    plt.plot(t[:], linewidth=0.5)
plt.show()

In [ ]:
import numpy as np
np.save("t-test-{}-{}-{}.npy".format(str(nb_traces), GADGET, ARCH), t_val)

In [ ]:
target.dis()
scope.dis()